In [ ]:
import numpy as np
import csv as csv
import json    
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import biosppy.signals.resp as resp
# Returns
#    -------
#    ts : array
#        Signal time axis reference (seconds).
#    filtered : array
#        Filtered Respiration signal.
#    zeros : array
#        Indices of Respiration zero crossings.
#    resp_rate_ts : array
#        Respiration rate time axis reference (seconds).
#    resp_rate : array
#        Instantaneous respiration rate (Hz).

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

### Read header

In [ ]:
# fname = 'https://github.com/malfarasplux/biofeatures/tree/master/riot_rip_acquisition/ak/opensignals_192.168.1.1008888_2020-02-05_11-54-42.txt'
fname = '../riot_rip_acquisition/ak/opensignals_192.168.1.1008888_2020-02-05_11-54-42.txt'
with open(fname) as datafile:
    readdata = csv.reader(datafile, delimiter='\t')
    header = [next(readdata), next(readdata), next(readdata)]
    header_json = json.loads(header[1][0][1:])
print(header_json)

### Read data

In [ ]:
data = np.genfromtxt(fname)
print(data.shape)

In [ ]:
# Locate the sampling rate 
srate = header_json[list(header_json.keys())[0]]['sampling rate']
srate

In [ ]:
# Locate the analog channel
col_num  = header_json[list(header_json.keys())[0]]['column'].index('ANALOG1')
col_num

In [ ]:
ripdata = data[:,col_num]

### Test the library

In [ ]:
import biofeatures

In [ ]:
B = biofeatures.breathing(ripdata[:2000])

In [ ]:
B.resp_intervals()

In [ ]:
B.resp_features()

In [ ]:
B.features

In [ ]:
B.interval_breathe_in

In [ ]:
B.interval_lengths

In [ ]:
np.sum(B.interval_lengths[B.interval_breathe_in])/B.srate

In [ ]:
np.sum(B.interval_lengths[np.logical_not(B.interval_breathe_in)])/B.srate

In [ ]:
B.feature_names

In [ ]:
B.resp_rate*60

### Simulate real time

In [ ]:
import time

In [ ]:
ti = time.time()

In [ ]:
tf = time.time()
ind = 10000
ts = 0.5

# 5 seconds, every 0.5
while tf-ti < 5:
    B.data = ripdata[ind-B.buffer_length:ind]
    ind += int(ts*B.srate)
    B.resp_intervals()
    B.resp_features()
    print(B.features)
    time.sleep(ts)
    tf = time.time()


In [ ]:
B.buffer_length


In [ ]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import time


In [ ]:
riot_ip = '192.168.0.102'
riot_port = 31000
actuator_port = 12000
actuator_ip = '192.168.0.103'

client = SimpleUDPClient(riot_ip, riot_port) 
resp_data = []

In [ ]:
def process_riot_data(unused_addr, *values):
    global resp_data, last_update, client
    global t0, B
    
        
    new_data = values[12]
    resp_data.append(new_data)
    if B.WU:
        B.data = resp_data[-B.buffer_length:]
#     print(new_data)

    warmup = 10
    tf = time.time()

#     dmax = 15
#     if tf-ti > 0.5 and tf-t0 < warmup+dmax and tf-t0 > warmup: 
#         print(tf-t0)
#         real_time_feat()
#         ti = time.time()


    if tf-t0 > warmup: 
        if not B.WU:
            print("WARMUP")
            B = biofeatures.breathing(data = resp_data)
            B.WU = True
            B.update_loop()

    
#     if len(resp_data) > 200*10 and time.time() - last_update > update_freq:
#         last_int, breathe_in = biofeatures.calc_resp_intervals(resp_data, last_breath = True)
                
#         if breathe_in:
#             print("Breathing in")
#             client.send_message("/actuator/inflate", 100.0)
#         else:
#             print("Breathing out")
#             client.send_message("/actuator/inflate", -100.0)
        
#         last_update = time.time()
    
    # only save the last 5 min of data
    if len(resp_data) > 200 * 60 * 5:
        resp_data = resp_data[-200*60*5:]

In [ ]:
t0 = time.time()
ti = time.time()
riot_dispatcher = dispatcher.Dispatcher()
riot_dispatcher.map("/*/raw", process_riot_data)

server = osc_server.ThreadingOSCUDPServer((riot_ip, riot_port), riot_dispatcher)
print("Serving on {}".format(server.server_address))
server.serve_forever()